# Periodic Mesh in GMSH

In [1]:
import gmsh

In [2]:
gmsh.initialize()

In [3]:
gmsh.model.add("Periodic x")

In [4]:
minCoords = (0., 0.)
maxCoords = (1.0, 1.0)
cellSize = 0.1

In [5]:
boundaries = {
   "Bottom": 1,
   "Top": 2,
   "Right": 3,
   "Left": 4,
}

In [6]:
xmin, ymin = minCoords
xmax, ymax = maxCoords

In [7]:
p1 = gmsh.model.geo.add_point(xmin,ymin,0., meshSize=cellSize)
p2 = gmsh.model.geo.add_point(xmax,ymin,0., meshSize=cellSize)
p3 = gmsh.model.geo.add_point(xmin,ymax,0., meshSize=cellSize)
p4 = gmsh.model.geo.add_point(xmax,ymax,0., meshSize=cellSize)

l1 = gmsh.model.geo.add_line(p1, p2, tag=boundaries["Bottom"])
l2 = gmsh.model.geo.add_line(p2, p4, tag=boundaries["Right"])
l3 = gmsh.model.geo.add_line(p4, p3, tag=boundaries["Top"])
l4 = gmsh.model.geo.add_line(p3, p1, tag=boundaries["Left"])

cl = gmsh.model.geo.add_curve_loop((l1, l2, l3, l4))
surface = gmsh.model.geo.add_plane_surface([cl])

In [8]:
gmsh.model.geo.synchronize()

The periodicity transform is provided as a 4x4 affine transformation matrix, given by row.
During mesh generation, the mesh on the right side will be created by copying the mesh from the left side.

To impose that the right side should match the left side, the following periodicity constraint is set:

$M = \begin{pmatrix}
  1 & 0 & 0 & 1\\ 
  0 & 1 & 0 & 0\\
  0 & 0 & 1 & 0\\
  0 & 0 & 0 & 1
\end{pmatrix}$

The left side is copied by translating it along the x axis.

Such a 4 by 4 matrix M corresponds to a afﬁne transformation T() that transforms point (or vector) x to point (or vector) y. The upper-left 3 × 3 sub-matrix of the matrix shown above represents a rotation transform, but may also include scales and shears. The last column of the matrix represents a translation. When used as a coordinate system, the upper-left 3 x 3 sub-matrix represents an orientation in space while the last column vector represents a position in space. The transformation T() of point x to point y is obtained by performing the matrix-vector multiplication:

$ y = Mx $ 

see for more information:

https://www.brainvoyager.com/bv/doc/UsersGuide/CoordsAndTransforms/SpatialTransformationMatrices.html#:~:text=The%204%20by%204%20transformation,in%20the%20first%20three%20columns.

In [9]:
translation = [1, 0, 0, 1, 
               0, 1, 0, 0,
               0, 0, 1, 0, 
               0, 0, 0, 1]

In [10]:
gmsh.model.mesh.setPeriodic(1, [boundaries["Right"]], [boundaries["Left"]], translation)

In [11]:
# Add Physical groups
for name, tag in boundaries.items():
    gmsh.model.add_physical_group(1, [tag] , tag)
    gmsh.model.set_physical_name(1, tag, name)

gmsh.model.addPhysicalGroup(2, [surface], surface)
gmsh.model.setPhysicalName(2, surface, "Elements")

In [12]:
gmsh.model.mesh.get_periodic(1, [boundaries["Right"]])

array([4], dtype=int32)

In [13]:
gmsh.model.mesh.get_periodic(1, [boundaries["Left"]])

array([4], dtype=int32)

In [14]:
#gmsh.fltk.run()

In [15]:
gmsh.model.mesh.generate(2) 
gmsh.write("periodicx.msh")
gmsh.finalize()

Info    : Reconstructing periodicity for curve connection 3 - 4
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : [100%] Meshing curve 3 (Line) as a copy of curve 4
Info    : Done meshing 1D (Wall 0.00341694s, CPU 0.005503s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.022048s, CPU 0.019632s)
Info    : Reconstructing periodicity for curve connection 3 - 4
Info    : 142 nodes 286 elements
Info    : Writing 'periodicx.msh'...
Info    : Done writing 'periodicx.msh'


## Importing in PETSc

Could be useful:
https://github.com/firedrakeproject/firedrake/issues/1852

In [16]:
from petsc4py import PETSc

In [17]:
plex = PETSc.DMPlex().createFromFile("periodicx.msh")

In [19]:
plex.view()

DM Object: unnamed 1 MPI process
  type: plex
unnamed in 2 dimensions:
  Number of 0-cells per rank: 131
  Number of 1-cells per rank: 373
  Number of 2-cells per rank: 242
Periodic mesh coordinates localized
Labels:
  celltype: 3 strata with value/size (0 (131), 3 (242), 1 (373))
  depth: 3 strata with value/size (0 (131), 1 (373), 2 (242))
  Cell Sets: 1 strata with value/size (1 (242))
  Face Sets: 4 strata with value/size (1 (10), 2 (10), 3 (10), 4 (10))
